 ## Protoype of KNN model used in this paper: https://www.sciencedirect.com/science/article/abs/pii/S0950705116000708?fbclid=IwAR3IMjfICBRERZVN_nopdLX3uYXHOMfwQgm-Lb7v_uvJ6w6J1_ixNzcelX8

In [1]:
## Set environment variable for data reading
import os
os.environ["PTB_FOLDER_PATH"] = "/Users/cesargenatios/Downloads/ptb-diagnostic-ecg-database-1.0.0"

In [10]:
## Load data from PTB dataset
import sys
sys.path.insert(0,"/Users/cesargenatios/Desktop/cappy/ptd_data_handler")
from data_formatter import *
all_patient_data = get_formatted_ptb_data()


## Data Loading

In [57]:
## Find a healthy control
for patient in all_patient_data.keys():
    for recording in all_patient_data[patient].keys():
        if all_patient_data[patient][recording].label[0] == "Healthy control":
            print(patient, recording)


patient229 s0453_re
patient229 s0452_re
patient245 s0474_re
patient245 s0480_re
patient242 s0471_re
patient243 s0472_re
patient244 s0473_re
patient104 s0306lre
patient166 s0275lre
patient150 s0287lre
patient105 s0303lre
patient169 s0329lre
patient169 s0328lre
patient156 s0299lre
patient180 s0477_re
patient180 s0545_re
patient180 s0374lre
patient180 s0475_re
patient180 s0490_re
patient180 s0476_re
patient180 s0561_re
patient174 s0325lre
patient174 s0324lre
patient174 s0300lre
patient173 s0305lre
patient116 s0302lre
patient172 s0304lre
patient117 s0292lre
patient117 s0291lre
patient121 s0311lre
patient266 s0502_re
patient235 s0461_re
patient251 s0486_re
patient251 s0503_re
patient251 s0506_re
patient260 s0496_re
patient267 s0504_re
patient233 s0458_re
patient233 s0483_re
patient233 s0482_re
patient233 s0457_re
patient233 s0459_re
patient234 s0460_re
patient241 s0469_re
patient241 s0470_re
patient279 s0531_re
patient279 s0532_re
patient279 s0534_re
patient279 s0533_re
patient246 s0478_re


In [58]:
## Choose two patients one with MI, one with Non-MI to test PoC. Test on another 
mi_train_df = all_patient_data['patient024']['s0094lre']
healthy_train_df = all_patient_data['patient252']['s0487_re']

mi_test_df = all_patient_data['patient024']['s0083lre']

In [62]:
## Delete rest of data to free up space
del all_patient_data

## Data Processing

In [ ]:
## Resample to 1000 Hz

## Use db6 for baseline wander removal

## Pan Tompkins to detect R peaks, select 250 left of R and 500 right of R as one beat

## DWT on each beat to the 4th power

## Feature Extraction

In [ ]:
## Extract 12 features on each of the reconstructed signals to the 4 power
## Each signal will have 48 features

## Approximate Entropy
## Signal Energy
## Fuzzy Entropy
## Kolmogorov-SInai Entropy
## Permutation Entropy
## Shannon Entropy
## Renyi Entropy
## Tsallis Entropy
## Wavelet Entropy
## Fractal Dimension
## Kolmogorov Complexity
## Largest Lyapuniv Exponent

## Prepare data for training

In [ ]:
## Standardize Data
## Separate data and labels
## Separate train and testing

## Model architecture

In [63]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)

## Model training

In [66]:
mi_train_df.columns

Index(['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6', 'vx', 'vy', 'vz', 'patient_id', 'recording_id', 'label',
       'acute_infarction_localization', 'former_infarction_localization'],
      dtype='object')

In [70]:
model.fit(mi_train_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']], mi_train_df["label"])
y_pred = model.predict(mi_test_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']])

## Metrics and model validation

In [73]:
import sklearn.metrics
accuracy = sklearn.metrics.accuracy_score(mi_test_df['label'].to_numpy(), y_pred)
precision = sklearn.metrics.precision_score(mi_test_df['label'].to_numpy(), y_pred)
recall = sklearn.metrics.recall_score(mi_test_df['label'].to_numpy(), y_pred)
print("Accuracy: {accuracy}, Precision: {precision}, Recall:{recall}".format(accuracy=accuracy, precision=precision, recall=recall))

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1037: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


Accuracy: 1.0, Precision: 0.0, Recall:0.0


In [74]:
print(model.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [78]:
test_inference = mi_test_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']]
test_inference = test_inference.head(1).to_numpy()

array([[ 0.4205,  0.1845, -0.236 , -0.302 ,  0.328 , -0.026 , -0.108 ,
         0.0065,  0.083 , -0.0705,  0.214 ,  0.168 ]])

In [79]:
pred_inference = model.predict(test_inference)

In [80]:
import pickle
pkl_filename = "knn_lfp.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)